In [1]:
!pip install --quiet vit-keras

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from vit_keras import vit

In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/train/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
vit_base_16 = vit.vit_b16(
        image_size = (128,128),
        activation = 'softmax',
        pretrained = None,
        include_top = False,
        pretrained_top = False,
        classes = 1)

vit_base_16_model = tf.keras.Sequential([
        vit_base_16,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, 'sigmoid')
    ])

In [6]:
vit_base_16_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b16 (Functional)        (None, 768)               85697280  
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 1)                 769       
                                                                 
Total params: 85,698,049
Trainable params: 85,698,049
Non-trainable params: 0
_________________________________________________________________


In [7]:
METRICS = [
        'accuracy',
    ]
    
vit_base_16_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=METRICS
    )

In [8]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [9]:
history = vit_base_16_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)

Epoch 1/10
652/652 [==============================] - 204s 240ms/step - loss: 0.6823 - accuracy: 0.7199 - val_loss: 0.6976 - val_accuracy: 0.6234 - lr: 0.0100
Epoch 2/10
652/652 [==============================] - 120s 184ms/step - loss: 0.5822 - accuracy: 0.7423 - val_loss: 0.6789 - val_accuracy: 0.6250 - lr: 0.0089
Epoch 3/10
652/652 [==============================] - 121s 185ms/step - loss: 0.5590 - accuracy: 0.7395 - val_loss: 0.6639 - val_accuracy: 0.6250 - lr: 0.0079
Epoch 4/10
652/652 [==============================] - 121s 185ms/step - loss: 0.5513 - accuracy: 0.7458 - val_loss: 0.7913 - val_accuracy: 0.6154 - lr: 0.0071
Epoch 5/10
652/652 [==============================] - 120s 184ms/step - loss: 0.5527 - accuracy: 0.7481 - val_loss: 0.6909 - val_accuracy: 0.6010 - lr: 0.0063
Epoch 6/10
652/652 [==============================] - 120s 184ms/step - loss: 0.5535 - accuracy: 0.7448 - val_loss: 0.7368 - val_accuracy: 0.6250 - lr: 0.0056
Epoch 7/10
652/652 [==========================

In [10]:
vit_base_16_model.save("ViT_xray_Pneumonia.h5")